In [ ]:
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from simbio import Constant, Parameter, Simulator, Species

from corbat import ARM


def get_default(x: Species | Parameter | Constant):
    try:
        return x.variable.initial
    except Exception:
        return x.default


sim = Simulator(ARM, backend="numba")
t = np.linspace(0, 15 * 3600, 1_000)
df0 = sim.solve(save_at=t)
df0.index /= 3600
to_plot = list(map(str, [ARM.C3_A, ARM.C8_A, ARM.Apop]))
onset0 = df0[to_plot].diff().idxmax()
onset0

In [ ]:
def to_widget(x, /):
    value = get_default(x)
    offset = np.log10(value)
    return x, ipywidgets.FloatLogSlider(value, min=offset - 5, max=offset + 5)


def plot(df, /):
    df = df[to_plot]
    df.index /= 3600
    df = df.diff()
    onset_diff = df.idxmax() - onset0
    df.plot(title=str(onset_diff))


sim.interact(
    save_at=t,
    values=dict(
        map(
            to_widget,
            [
                ARM.L,
                ARM.R,
                ARM.mitocondria_volume_fraction,
                ARM.pore_transport_rate,
                ARM.transloc_rate,
                ARM.Bcl2,
                ARM.CytoC_M,
                ARM.Smac_M,
                ARM.Mito_I,
            ],
        )
    ),
    func=plot,
)

In [ ]:
t = np.linspace(0, 15 * 3600, 1_000)


def change(X: Species):
    values = np.logspace(-5, 5, 50)

    def C3_max(v):
        return sim.solve({X: v}, save_at=t)[str(ARM.C3_A)].diff().idxmax()

    initial = get_default(X)
    t_max = np.fromiter(map(C3_max, initial * values), dtype=float)
    t_max /= 3600
    return pd.Series(t_max, values, name=str(X))


df = pd.concat(
    map(
        change,
        [
            ARM.L,
            ARM.R,
        ],
    ),
    axis=1,
)
df.plot(
    logx=True,
    figsize=(6, 3),
    ylabel="Onset (hour)",
    xlabel="Change from model default",
    ylim=(0, None),
)